# 🚀 Whisper GPU vs CPU 速度对比测试

测试 GPU 加速对 Whisper 字幕提取的性能提升

## 1️⃣ 环境设置

In [ ]:
# 安装依赖
!pip install -q openai-whisper yt-dlp

import torch
import whisper
import time
from pathlib import Path

print(f"\n🔥 GPU 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU 名称: {torch.cuda.get_device_name(0)}")
    print(f"   GPU 显存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 2️⃣ 下载测试音频

In [ ]:
# 使用一个短视频测试（B站短视频）
# 你可以替换成任意视频链接
TEST_URL = "https://www.bilibili.com/video/BV1GJ411x7h7"  # @param {type:"string"}

import yt_dlp

print("下载测试音频...")
ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': 'test_audio.%(ext)s',
    'quiet': True,
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(TEST_URL, download=True)
    duration = info.get('duration', 0)
    title = info.get('title', 'test')

audio_file = list(Path('.').glob('test_audio.*'))[0]
print(f"\n音频: {audio_file}")
print(f"时长: {duration}秒 ({duration/60:.1f}分钟)")

## 3️⃣ CPU vs GPU 速度对比

In [ ]:
# 测试模型
MODEL_SIZE = "small"  # @param ["tiny", "base", "small", "medium"]

results = {}

# ============ CPU 测试 ============
print("\n" + "="*60)
print("🖥️  CPU 模式测试")
print("="*60)

import gc
gc.collect()

load_start = time.time()
model_cpu = whisper.load_model(MODEL_SIZE, device="cpu")
load_time_cpu = time.time() - load_start
print(f"模型加载: {load_time_cpu:.2f}秒")

transcribe_start = time.time()
result_cpu = model_cpu.transcribe(str(audio_file), language='zh', verbose=False)
transcribe_time_cpu = time.time() - transcribe_start
print(f"转录耗时: {transcribe_time_cpu:.2f}秒")
print(f"速度比: {duration/transcribe_time_cpu:.2f}x 实时")

del model_cpu
gc.collect()

results['cpu'] = {
    'load': load_time_cpu,
    'transcribe': transcribe_time_cpu,
    'total': load_time_cpu + transcribe_time_cpu,
    'speed_ratio': duration / transcribe_time_cpu
}

# ============ GPU 测试 ============
if torch.cuda.is_available():
    print("\n" + "="*60)
    print("🚀 GPU 模式测试")
    print("="*60)
    
    torch.cuda.empty_cache()
    gc.collect()
    
    load_start = time.time()
    model_gpu = whisper.load_model(MODEL_SIZE, device="cuda")
    load_time_gpu = time.time() - load_start
    print(f"模型加载: {load_time_gpu:.2f}秒")
    
    transcribe_start = time.time()
    result_gpu = model_gpu.transcribe(str(audio_file), language='zh', verbose=False)
    transcribe_time_gpu = time.time() - transcribe_start
    print(f"转录耗时: {transcribe_time_gpu:.2f}秒")
    print(f"速度比: {duration/transcribe_time_gpu:.2f}x 实时")
    
    del model_gpu
    torch.cuda.empty_cache()
    
    results['gpu'] = {
        'load': load_time_gpu,
        'transcribe': transcribe_time_gpu,
        'total': load_time_gpu + transcribe_time_gpu,
        'speed_ratio': duration / transcribe_time_gpu
    }
else:
    print("\n⚠️ GPU 不可用，跳过 GPU 测试")
    print("   请在: 运行时 → 更改运行时类型 → 启用 GPU")

## 4️⃣ 测试结果对比

In [ ]:
print("\n" + "="*60)
print("📊 测试结果对比")
print("="*60)

print(f"\n测试音频时长: {duration}秒 ({duration/60:.1f}分钟)")
print(f"Whisper 模型: {MODEL_SIZE}")

if 'cpu' in results:
    print(f"\n🖥️  CPU:")
    print(f"   模型加载: {results['cpu']['load']:.2f}秒")
    print(f"   转录耗时: {results['cpu']['transcribe']:.2f}秒")
    print(f"   总耗时: {results['cpu']['total']:.2f}秒")
    print(f"   速度比: {results['cpu']['speed_ratio']:.2f}x 实时")

if 'gpu' in results:
    print(f"\n🚀 GPU:")
    print(f"   模型加载: {results['gpu']['load']:.2f}秒")
    print(f"   转录耗时: {results['gpu']['transcribe']:.2f}秒")
    print(f"   总耗时: {results['gpu']['total']:.2f}秒")
    print(f"   速度比: {results['gpu']['speed_ratio']:.2f}x 实时")
    
    # 计算加速比
    speedup = results['cpu']['transcribe'] / results['gpu']['transcribe']
    time_saved = results['cpu']['transcribe'] - results['gpu']['transcribe']
    
    print(f"\n🏆 GPU 加速效果:")
    print(f"   转录加速: {speedup:.2f}x")
    print(f"   时间节省: {time_saved:.2f}秒 ({time_saved/duration*100:.1f}%)")
    
    # 推算长视频时间
    for video_len in [10, 30, 60]:  # 分钟
        seconds = video_len * 60
        cpu_time = seconds / results['cpu']['speed_ratio']
        gpu_time = seconds / results['gpu']['speed_ratio']
        print(f"   {video_len}分钟视频: CPU {cpu_time/60:.1f}分钟 vs GPU {gpu_time/60:.1f}分钟")

## 5️⃣ 验证结果一致性

In [ ]:
# 验证 CPU 和 GPU 结果是否一致
if 'cpu' in results and 'gpu' in results:
    print("\n验证结果一致性...")
    
    # 对比文本长度
    cpu_len = len(result_cpu['text'])
    gpu_len = len(result_gpu['text'])
    print(f"文本长度: CPU {cpu_len} vs GPU {gpu_len}")
    
    # 对比段落数
    cpu_segments = len(result_cpu['segments'])
    gpu_segments = len(result_gpu['segments'])
    print(f"段落数量: CPU {cpu_segments} vs GPU {gpu_segments}")
    
    # 对比前3个段落
    print("\n前3个段落对比:")
    for i in range(min(3, cpu_segments)):
        cpu_text = result_cpu['segments'][i]['text'].strip()
        gpu_text = result_gpu['segments'][i]['text'].strip()
        match = "✅" if cpu_text == gpu_text else "⚠️"
        print(f"  [{i+1}] {match}")
        print(f"      CPU: {cpu_text}")
        print(f"      GPU: {gpu_text}")
    
    print("\n结论: CPU 和 GPU 结果基本一致，差异可能来自浮点运算精度")

## 📝 预期结果

| 项目 | CPU | GPU | 加速比 |
|------|-----|-----|--------|
| 模型加载 | ~3秒 | ~2秒 | 1.5x |
| 转录速度 | ~5-10x实时 | ~30-50x实时 | **5-10x** |

### 典型场景推算（small 模型，Colab T4 GPU）:

| 视频时长 | CPU 耗时 | GPU 耗时 |
|----------|----------|----------|
| 5分钟 | ~1分钟 | ~10秒 |
| 10分钟 | ~2分钟 | ~20秒 |
| 30分钟 | ~6分钟 | ~1分钟 |
| 60分钟 | ~12分钟 | ~2分钟 |